# Image Processing

In [1]:
pwd

'/home/wsuser/work'

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
train_datagen = ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True,rotation_range=180,vertical_flip=False)

In [4]:
test_datagen = ImageDataGenerator(rescale=1/255)

In [5]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ae9f436af13b481c838e71e37c053886 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='tlJajZHIt8NmWfBL5PkK8BZkFdh-XujKMBiFVxqNKDGu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_2 = client_ae9f436af13b481c838e71e37c053886.get_object(Bucket='emergingforestfiredetection-donotdelete-pr-fk8k9ghyqf0ib6', Key='Dataset.zip')['Body']
# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [6]:
from io import BytesIO
import zipfile 
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
pwd

'/home/wsuser/work'

In [8]:
import os
filenames=os.listdir('/home/wsuser/work/Dataset/train_set')

In [9]:
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/train_set',target_size=(128,128),class_mode = 'binary',batch_size=32)

Found 436 images belonging to 2 classes.


In [10]:
x_test=test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/test_set',target_size=(128,128),class_mode = 'binary',batch_size=32)

Found 121 images belonging to 2 classes.


In [11]:
x_train.class_indices

{'forest': 0, 'with fire': 1}

# MODEL BUILDING

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
model = Sequential()

In [15]:
#add convolutional layer
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [16]:
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [17]:
#add flatten layer
model.add(Flatten())

In [18]:
#add hidden layer
model.add(Dense(kernel_initializer='uniform',activation='relu',units=150))

In [19]:
#add hidden layer
model.add(Dense(units=64,kernel_initializer='uniform',activation='relu'))

In [20]:
#add output layer
model.add(Dense(kernel_initializer='uniform',activation='sigmoid',units=1))

In [21]:
print(x_train.class_indices)

{'forest': 0, 'with fire': 1}


In [22]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 150)               19051350  
                                                                 
 dense_1 (Dense)             (None, 64)                9664      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                        

In [24]:
model.fit_generator(x_train,steps_per_epoch=14,epochs=10,validation_data=x_test,validation_steps=4)

Epoch 1/10
14/14 [==============================] - 22s 2s/step - loss: 0.5460 - accuracy: 0.7110 - val_loss: 0.2490 - val_accuracy: 0.9008
Epoch 2/10
14/14 [==============================] - 20s 1s/step - loss: 0.2957 - accuracy: 0.8853 - val_loss: 0.1399 - val_accuracy: 0.9339
Epoch 3/10
14/14 [==============================] - 20s 1s/step - loss: 0.2964 - accuracy: 0.8739 - val_loss: 0.1181 - val_accuracy: 0.9504
Epoch 4/10
14/14 [==============================] - 21s 1s/step - loss: 0.2385 - accuracy: 0.9014 - val_loss: 0.1022 - val_accuracy: 0.9752
Epoch 5/10
14/14 [==============================] - 20s 1s/step - loss: 0.1972 - accuracy: 0.9151 - val_loss: 0.0878 - val_accuracy: 0.9752
Epoch 6/10
14/14 [==============================] - 20s 1s/step - loss: 0.1916 - accuracy: 0.9266 - val_loss: 0.1730 - val_accuracy: 0.9091
Epoch 7/10
14/14 [==============================] - 20s 1s/step - loss: 0.2449 - accuracy: 0.8968 - val_loss: 0.0702 - val_accuracy: 0.9917
Epoch 8/10
14/14 [==

In [25]:
model.save('forestfire.h5')

In [26]:
!tar -zcvf Emerging-Forest-Fires.tgz forestfire.h5 

forestfire.h5


In [27]:
ls -1

Dataset/
Emerging-Forest-Fires.tgz
forestfire.h5


In [28]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 28.0 MB/s eta 0:00:01


In [29]:
from ibm_watson_machine_learning import APIClient
wml_credentials= {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"9gsp3sMazuRfe8UsjUSUtPhXMGK_CL_FvmHWg4mtrNC4"
}
client = APIClient(wml_credentials)

#9gsp3sMazuRfe8UsjUSUtPhXMGK_CL_FvmHWg4mtrNC4

In [30]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [31]:
space_uid = guid_from_space_name(client,'Forest-fires')
print("Space UID =" +space_uid)

Space UID =a1d7a045-d7da-4eb3-8f12-b12cbf1507a3


In [32]:
client.set.default_space(space_uid)

'SUCCESS'

In [33]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [36]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [37]:
model_details = client.repository.store_model(model='Emerging-Forest-Fires.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.5",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.5 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html
This method is deprecated, please use get_model_id()


In [38]:
model_id

'd18254cf-ec4a-47a5-a90c-f803b23e50bb'

In [39]:
client.repository.download(model_id, 'Emerging_forest_fire_detection.tar.gz')

Successfully saved model content to file: 'Emerging_forest_fire_detection.tar.gz'


'/home/wsuser/work/Emerging_forest_fire_detection.tar.gz'